# 2. Create an Incident from an Alert

Notebook to build all of the different stix objects associated with a Phishing Alert imitialising an Incident. Note that the Stix objects created by these classes are frozen dicts, so its better to create the components and then the whole, rather than create an empty whole first and then add the components, which requires updating.

We take v3.0 of the [Incident Core Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/incident-core/Incident%20Extension%20Suite.adoc) specification, with particular focus on the Sighting Extensions for Evidence, and combine it with the standard Stix approach. There are six stages:

1. First setup the global parameters and retrieve the context memory
2. Create the objects from the phishing email Alert,
2. Add these objects to the `ObservedData` object
3. Then create the `Indicator` and get the user `Identity` object from context memory
4. Create the `Sighting` object with the `SightingAlert` extension
5. Create the Event and the Sequence objects
6. Create the Incident, with the [Incident Core Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/incident-core/Incident%20Extension%20Suite.adoc), and add all of the objects to it


Each time a block makes an object, and saves it as a json, this notebook will parse the object into an actual Stix object, so it can be bundled and printy printed (This step also verifies the objects are created correctly, and is cool).

## A. Load Imports

### A.1 StixORM Imports


In [1]:
import sys
!{sys.executable} -m pip install stixorm
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle, Incident, URL, EmailMessage
)
from stixorm.module.definitions.os_threat import (
    EventCoreExt, Event, SocialMediaContact, ContactNumber, IncidentCoreExt, TaskCoreExt,
    Task, SightingEvidence, Sequence, SequenceExt, AnecdoteExt, Anecdote,
    SightingAnecdote, SightingAlert, SightingContext, SightingExclusion,
    SightingEnrichment, SightingHunt, SightingFramework, SightingExternal
)
from stixorm.module.authorise import import_type_factory
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects
import_type = import_type_factory.get_all_imports()
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

### A.2 Relative Path Imports
https://stackoverflow.com/questions/76162459/jupyter-notebook-importerror-attempted-relative-import-with-no-known-parent-pac/77528726#77528726

In [2]:
import sys
sys.path.append('../')
import os

### A.3 Relative Import of Blocks

In [3]:
import json
from Block_Families.Objects.SCO.URL.make_url import main as make_url
from Block_Families.Objects.SCO.Email_Addr.make_email_addr import main as make_email_addr
from Block_Families.Objects.SCO.Email_Message.make_email_msg import main as make_email_msg
from Block_Families.Objects.SRO.Relationship.make_sro import main as make_sro
from utilities import emulate_ports, unwind_ports, conv

context_base = "../Orchestration/Context_Mem/"
path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_derived.json"
alert_data = {
    "from": "evil@northkorea.nk",
    "to": "naive@example.com",
    "reporter": "nsmith",
    "url": "https://www.northkorea.nk/we/are/mad/"
}
alert_details = {
    "alert_name": "Suspcious Email",
    "alert_source": "user-report",
    "alert_log": "I have found a suspicious email"
}
# Make the Variable to hold the Bundle List
bundle_list = []
sequence_start_refs = []
sequence_refs = []
task_refs = []
event_refs = []
impact_refs = []
other_object_refs = []
results_paths = {}

### A.4 Load the Context Memory

In [4]:
# 1. Set the Variables
context_identities = []
identities_file = "identity_bundle.json"
context_path = context_base + identities_file
#2. Load the file
if os.path.exists(context_path):
    with open(context_path, "r") as script_input:
        export_data = json.load(script_input)
        context_identities = export_data["objects"]

## B. Create the Objects to Document the Phishing Alert

### B.1 Create the Email Address for the Threat Actor

In [5]:
def local_make_threat_email_addr(email_path, results_path):
    #
    # 1. Set the Relative Input and Output Paths for the block
    email_data_rel_path = path_base + email_path
    email_results_rel_path = results_base + results_path
    #
    # 2. Make the Email Address object
    make_email_addr(email_data_rel_path,email_results_rel_path)
    # 3. Retrieve the saved file
    if os.path.exists(email_results_rel_path):
        with open(email_results_rel_path, "r") as script_input:
            export_data = json.load(script_input)
            export_data_list = export_data["email-addr"]
            stix_object = export_data_list[0]
            # 4. convert it into a Stix Object and append to the bundle
            email_addr = EmailAddress(**stix_object)
            print(email_addr.serialize(pretty=True))
            bundle_list.append(email_addr)

email_path = "SCO/Email_Addr/email_addr_THREAT.json"
results_path = "THREAT__email_addr.json"
# make the email object
local_make_threat_email_addr(email_path, results_path)
# recover the threat email object in dict form
threat_email_addr = conv(bundle_list[0])

{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212",
    "value": "evil@northkorea.nk",
    "display_name": "Bad Man"
}


### B.2 Retrieve the User Account, the Email Address, and the TR Identity from the Context Memory

In [6]:
# For Naive Smith, find the user account and the email. Plus get my own identity
reporter_name = alert_data["reporter"]
to_email_addr = alert_data["to"]
TR_name = "Trusty Jones"
# Setup variables to hold the dicts
reporter_account = {}
reporter_email_addr = {}
TR_identity = {}
# scroll through the context to find the user account
for rec in context_identities:
    if rec["type"] == "user-account":
        if rec["account_login"] == reporter_name:
            reporter_account = rec
    if rec["type"] == "email-addr":
        if rec["value"] == to_email_addr:
            reporter_email_addr = rec
    if rec["type"] == "identity":
        if rec["name"] == TR_name:
            TR_identity = rec

### B.3 Make the URL

In [7]:
def local_make_url(url_path, results_path, hyperlink=None):
    #
    # 1. Set the Relative Input and Output Paths for the block
    url_data_rel_path = path_base + url_path
    url_results_rel_path = results_base + results_path
    #
    # NOTE: This code is only To fake input ports
    # Add the User Account object and the  EmailAddress
    #  Form data file
    #
    if os.path.exists(url_data_rel_path):
        with open(url_data_rel_path, "r") as sro_form:
            results_data = json.load(sro_form)
            results_data["hyperlink"] = hyperlink
        with open(url_data_rel_path, 'w') as f:
            f.write(json.dumps(results_data))
    #
    # Make the Email Address object
    #
    make_url(url_data_rel_path,url_results_rel_path)
    #
    # Remove Port Emulation - Fix the data file so it only has form data
    #
    #unwind_ports(url_data_rel_path)
    # 2. Make the Email Address object
    make_url(url_data_rel_path,url_results_rel_path)
    # 3. Retrieve the saved file
    if hyperlink:
        if os.path.exists(url_results_rel_path):
            with open(url_results_rel_path, "r") as script_input:
                export_data = json.load(script_input)
                export_data_list = export_data["url"]
                stix_object = export_data_list[0]
                # 4. convert it into a Stix Object and append to the bundle
                url_object = URL(**stix_object)
                print(url_object.serialize(pretty=True))
                bundle_list.append(url_object)

url_path = "SCO/URL/suspicious_url.json"
results_path = "THREAT__url.json"
hyperlink = alert_data["url"]
# make the email object
local_make_url(url_path, results_path, hyperlink)
# recover the threat email object in dict form
sus_url = conv(bundle_list[1])

{
    "type": "url",
    "spec_version": "2.1",
    "id": "url--3279c7de-8f91-5c1a-99d9-d6546c6c41f7",
    "value": "https://www.northkorea.nk/we/are/mad/"
}


### B.4 Make the Email Message

In [8]:
def local_make_msg(msg_path, results_path, from_ref=None, to_refs=None, cc_refs=None, bcc_refs=None):
    #
    # 1. Set the Relative Input and Output Paths for the block
    msg_data_rel_path = path_base + msg_path
    msg_results_rel_path = results_base + results_path
    #
    # NOTE: This code is only To fake input ports
    # Add the User Account object and the  EmailAddress
    #  Form data file
    #
    if os.path.exists(msg_data_rel_path):
        with open(msg_data_rel_path, "r") as sro_form:
            results_data = json.load(sro_form)
            if from_ref:
                results_data["from_ref"] = from_ref
            if to_refs or to_refs != []:
                results_data["to_refs"] = to_refs
            if cc_refs or cc_refs != []:
                results_data["cc_refs"] = cc_refs
            if bcc_refs or bcc_refs != []:
                results_data["bcc_refs"] = bcc_refs
        with open(msg_data_rel_path, 'w') as f:
            f.write(json.dumps(results_data))
    #
    # Make the Email Msg object
    #
    make_email_msg(msg_data_rel_path,msg_results_rel_path)
    #
    # Remove Port Emulation - Fix the data file so it only has form data
    #
    #unwind_ports(msg_data_rel_path)
    # 3. Retrieve the saved file
    if os.path.exists(msg_results_rel_path):
        with open(msg_results_rel_path, "r") as script_input:
            export_data = json.load(script_input)
            export_data_list = export_data["email-message"]
            stix_object = export_data_list[0]
            # 4. convert it into a Stix Object and append to the bundle
            msg_object = EmailMessage(**stix_object)
            print(msg_object.serialize(pretty=True))
            bundle_list.append(msg_object)

msg_path = "SCO/Email_Message/suspicious_email_msg.json"
results_path = "SUSS__email_msg.json"
from_ref = threat_email_addr
to_ref = [reporter_email_addr]
cc_ref = []
bcc_ref = []
# make the email object
local_make_msg(msg_path, results_path, from_ref, to_ref, cc_ref, bcc_ref)
# recover the threat email object in dict form
sus_msg = conv(bundle_list[2])

<class 'dict'>
{'type': 'email-addr', 'spec_version': '2.1', 'id': 'email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212', 'value': 'evil@northkorea.nk', 'display_name': 'Bad Man'}
{
    "type": "email-message",
    "spec_version": "2.1",
    "id": "email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d",
    "is_multipart": false,
    "date": "2020-10-19T01:01:01Z",
    "from_ref": "email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212",
    "to_refs": [
        "email-addr--4722424c-7012-56b0-84d5-01d076fc547b"
    ],
    "subject": "we are coming for you",
    "body": "some bad stuff written here"
}


### B.4 Make the "derived-from" SRO